In [8]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [10]:
tf.keras.backend.clear_session()

In [11]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [12]:
subject = 'Piauí - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [13]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PI.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Piauí - Produção de Cimento (t)',axis=1)
data

,Ano,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),...,Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - Consumo de Cimento (t)
0,2008,0.636377,7.951466,0.633132,0.744861,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,5.483335e+08,...,11.760983,0.670241,24.296582,1002.253734,1279.489487,28.374914,21.343931,67.378713,1.314599,449.076000
1,2009,0.638637,7.981121,0.637582,0.746332,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,5.302764e+08,...,9.472585,0.672829,23.274048,1002.162518,1397.889138,27.971629,21.269184,69.377054,1.242652,536.766000
2,2010,0.640114,7.946167,0.641655,0.747336,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,5.046425e+08,...,9.332485,0.674904,19.182853,1002.283003,1437.681759,27.813041,21.237580,70.062576,1.227247,639.535000
3,2011,0.640823,7.934312,0.645347,0.747880,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,4.854622e+08,...,10.985689,0.676479,18.107389,1002.395371,1443.014477,27.920185,21.009865,69.079510,1.247622,625.022000
4,2012,0.640158,9.098665,0.648074,0.747956,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,4.867661e+08,...,8.159014,0.677105,18.252844,1002.464093,1406.026571,27.983708,20.651339,67.998886,1.231493,687.536000
5,2013,0.640588,9.874811,0.652080,0.747535,2.552759e+07,1.456057e+06,6.451661,2.003921e+07,5.225848e+08,...,7.913561,0.678531,18.670752,1002.768122,1348.345803,27.951493,20.492168,67.705538,1.231771,778.779000
6,2014,0.638470,10.508945,0.654503,0.746830,2.597493e+07,1.458827e+06,6.481158,2.014649e+07,5.676413e+08,...,10.350442,0.678492,18.366943,1002.930152,1312.189559,28.101214,20.242493,66.500996,1.263497,755.713857
7,2015,0.636259,10.965401,0.655795,0.745732,2.623707e+07,1.443457e+06,6.481716,2.015777e+07,5.179392e+08,...,12.454398,0.677866,19.175926,1002.890504,1261.987570,28.208348,20.043769,65.528148,1.276249,794.945486
8,2016,0.648680,11.203809,0.670587,0.753641,2.645308e+07,1.433449e+06,6.489827,2.018916e+07,4.524144e+08,...,13.111908,0.689454,22.137114,1002.832338,1235.205362,28.178052,20.095966,65.401854,1.267215,842.687657
9,2017,0.648105,11.173036,0.673185,0.752518,2.658816e+07,1.433526e+06,6.502281,2.023043e+07,4.213689e+08,...,9.479812,0.689859,24.264986,1002.920008,1214.156503,28.068432,20.444122,66.544636,1.228392,831.356000


In [14]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.636377,7.951466,0.633132,0.744861,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,5.483335e+08,2.546644e+09,...,2.868108,11.760983,0.670241,24.296582,1002.253734,1279.489487,28.374914,21.343931,67.378713,1.314599
1,0.638637,7.981121,0.637582,0.746332,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,5.302764e+08,2.679945e+09,...,2.305885,9.472585,0.672829,23.274048,1002.162518,1397.889138,27.971629,21.269184,69.377054,1.242652
2,0.640114,7.946167,0.641655,0.747336,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,5.046425e+08,2.891930e+09,...,1.958246,9.332485,0.674904,19.182853,1002.283003,1437.681759,27.813041,21.237580,70.062576,1.227247
3,0.640823,7.934312,0.645347,0.747880,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,4.854622e+08,3.091988e+09,...,2.079088,10.985689,0.676479,18.107389,1002.395371,1443.014477,27.920185,21.009865,69.079510,1.247622
4,0.640158,9.098665,0.648074,0.747956,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,4.867661e+08,3.189506e+09,...,2.183282,8.159014,0.677105,18.252844,1002.464093,1406.026571,27.983708,20.651339,67.998886,1.231493
5,0.640588,9.874811,0.652080,0.747535,2.552759e+07,1.456057e+06,6.451661,2.003921e+07,5.225848e+08,3.093872e+09,...,2.403445,7.913561,0.678531,18.670752,1002.768122,1348.345803,27.951493,20.492168,67.705538,1.231771
6,0.638470,10.508945,0.654503,0.746830,2.597493e+07,1.458827e+06,6.481158,2.014649e+07,5.676413e+08,2.779340e+09,...,2.748397,10.350442,0.678492,18.366943,1002.930152,1312.189559,28.101214,20.242493,66.500996,1.263497
7,0.636259,10.965401,0.655795,0.745732,2.623707e+07,1.443457e+06,6.481716,2.015777e+07,5.179392e+08,2.561731e+09,...,2.501529,12.454398,0.677866,19.175926,1002.890504,1261.987570,28.208348,20.043769,65.528148,1.276249
8,0.648680,11.203809,0.670587,0.753641,2.645308e+07,1.433449e+06,6.489827,2.018916e+07,4.524144e+08,2.516859e+09,...,2.073464,13.111908,0.689454,22.137114,1002.832338,1235.205362,28.178052,20.095966,65.401854,1.267215
9,0.648105,11.173036,0.673185,0.752518,2.658816e+07,1.433526e+06,6.502281,2.023043e+07,4.213689e+08,2.497710e+09,...,2.177386,9.479812,0.689859,24.264986,1002.920008,1214.156503,28.068432,20.444122,66.544636,1.228392


In [15]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     536.766000
1     639.535000
2     625.022000
3     687.536000
4     778.779000
5     755.713857
6     794.945486
7     842.687657
8     831.356000
9     626.223000
10    702.437000
11    788.164000
12    826.368000
13    804.417000
14           NaN
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [16]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.244793,-1.286179,-1.543242,-1.364689,-1.791927,-2.430395,-1.983161,-1.934577,1.204752,-0.990234,...,2.026352,0.984445,-1.486495,1.532542,-1.329511,-0.499138,1.719528,1.574885,0.008389,2.526713
1,-0.733935,-1.264664,-1.229292,-0.796636,-1.615264,-1.276024,-1.702934,-1.710653,0.820657,-0.546688,...,0.085558,-0.051710,-1.104860,1.094793,-1.637011,0.971937,-0.703092,1.403767,1.371924,-0.247365
2,-0.399968,-1.290024,-0.941897,-0.409264,-1.098406,0.191239,-1.000708,-1.029511,0.275392,0.158672,...,-1.114490,-0.115145,-0.798779,-0.656658,-1.230840,1.466347,-1.655760,1.331414,1.839678,-0.841315
3,-0.239582,-1.298625,-0.681446,-0.199210,-0.616760,0.654083,-0.420225,-0.457837,-0.132596,0.824346,...,-0.697344,0.633403,-0.566581,-1.117067,-0.852032,1.532604,-1.012123,0.810102,1.168900,-0.055737
4,-0.389944,-0.453869,-0.489065,-0.170018,-0.190143,0.804775,-0.016723,-0.032922,-0.104860,1.148829,...,-0.337667,-0.646476,-0.474249,-1.054798,-0.620359,1.073042,-0.630530,-0.010676,0.431554,-0.677618
5,-0.292755,0.109237,-0.206415,-0.332297,0.201584,1.133256,0.361197,0.354549,0.657046,0.830615,...,0.422337,-0.757614,-0.264062,-0.875890,0.404563,0.356378,-0.824051,-0.375069,0.231393,-0.666887
6,-0.771591,0.569311,-0.035440,-0.604609,0.549114,1.310516,0.710469,0.697572,1.615453,-0.215960,...,1.613108,0.345772,-0.269828,-1.005951,0.950789,-0.092851,0.075353,-0.946654,-0.590506,0.556362
7,-1.271487,0.900476,0.055711,-1.028293,0.752766,0.327112,0.717082,0.733641,0.558230,-0.940033,...,0.760923,1.298413,-0.362089,-0.659624,0.817133,-0.716594,0.718932,-1.401597,-1.254312,1.048065
8,1.536902,1.073445,1.099309,2.024843,0.920581,-0.313247,0.813122,0.834017,-0.835563,-1.089341,...,-0.716758,1.596124,1.346608,0.608069,0.621045,-1.049353,0.536936,-1.282101,-1.340487,0.699728
9,1.406981,1.051119,1.282663,1.591118,1.025520,-0.308314,0.960590,0.965973,-1.495939,-1.153056,...,-0.358019,-0.048438,1.406313,1.519016,0.916592,-1.310878,-0.121575,-0.485063,-0.560729,-0.797166


In [17]:
reshaped_train_input = dfToInputRNN(train_input)

In [18]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 1.03720327,  0.88451686,  1.23879267,  0.33319037,
          0.82904731,  0.07834927,  0.64873804,  0.6622595 ,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042,  1

In [19]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     536.766000
1     639.535000
2     625.022000
3     687.536000
4     778.779000
5     755.713857
6     794.945486
7     842.687657
8     831.356000
9     626.223000
10    702.437000
11    788.164000
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [20]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    788.164
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [21]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.823567,0.785725,1.126392,-0.598225,0.529639,0.378729,0.253155,0.276615,1.108405,2.635823,...,-1.482779,-2.260447,0.91739,1.011919,0.418013,-0.843464,1.091606,-0.183281,-0.442768,-1.555022


In [22]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.82356725,  0.78572538,  1.12639184, -0.5982253 ,
          0.52963862,  0.37872907,  0.25315465,  0.27661526,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738,  0

In [23]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    826.368
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [24]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [25]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [27]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3742322158, 1790489810, 382394376, 1455717455, 2083586495, 2041779443, 1319907785, 4243389587, 3679361245, 1684287484, 3500880424, 3873286707, 4045689588, 2641906800, 3753648340, 1548597131, 4137692513, 3332658835, 1054878524, 3694792542, 66576371, 1074483797, 3220641678, 2759208643, 1818923793, 2983152839, 3863524189, 2447635714, 445433888, 2671764632, 237132999, 3734988920, 2905585473, 3293834619, 953849990, 1063809720, 3990948503, 802487000, 4275183341, 2575111067, 1647488979, 2235217694, 3683923026, 26577849, 1547614792, 3250962570, 1560433113, 3972706088, 1989265945, 3542048106, 2442020766, 1626130346, 3492524012, 3440783296, 3576526758, 2620494642, 2746529026, 3996737359, 984432993, 2582719575, 4057084340, 3061280431, 2604522227, 1652280862, 1661962368, 2966954642, 4067640397, 4233872231, 3109948447, 3560188646, 684689542, 597061133, 2995001426, 3727730935, 3983562579, 4032316994, 2471789554, 189767801, 1869990449, 3113590129, 1744101290, 2364647693, 3876098781, 3613908043, 2369

loss: 175.8815155029297


Step: 88 ___________________________________________
loss: 142.71783447265625


Step: 89 ___________________________________________
loss: 107.57064056396484


Step: 90 ___________________________________________
loss: 136.20045471191406


Step: 91 ___________________________________________
loss: 50.93058395385742
winner_seed: 3870812295


Step: 92 ___________________________________________
loss: 121.65706634521484


Step: 93 ___________________________________________
loss: 113.88541412353516


Step: 94 ___________________________________________
loss: 140.14804077148438


Step: 95 ___________________________________________
loss: 130.3017120361328


Step: 96 ___________________________________________
loss: 72.54301452636719


Step: 97 ___________________________________________
loss: 82.55072784423828


Step: 98 ___________________________________________
loss: 101.23339080810547


Step: 99 ___________________________________________
loss: 134.22682189941

In [28]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 523164.2188
Epoch 2/10000
1/1 [==============================] - 0s 7ms/step - loss: 511308.5938
Epoch 3/10000
1/1 [==============================] - 0s 8ms/step - loss: 481507.2188
Epoch 4/10000
1/1 [==============================] - 0s 7ms/step - loss: 270915.5938
Epoch 5/10000
1/1 [==============================] - 0s 8ms/step - loss: 14400.3408
Epoch 6/10000
1/1 [==============================] - 0s 8ms/step - loss: 12133.6123
Epoch 7/10000
1/1 [==============================] - 0s 9ms/step - loss: 6867.2075
Epoch 8/10000
1/1 [==============================] - 0s 8ms/step - loss: 5834.0063
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 6192.4868
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 7157.6685
Epoch 11/10000
1/1 [==============================] - 0s 5ms/step - loss: 5984.5630
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - loss: 10

1/1 [==============================] - 0s 4ms/step - loss: 1045.6117
Epoch 99/10000
1/1 [==============================] - 0s 5ms/step - loss: 1756.5555
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 687.0530
Epoch 101/10000
1/1 [==============================] - 0s 7ms/step - loss: 439.0151
Epoch 102/10000
1/1 [==============================] - 0s 5ms/step - loss: 1362.1750
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 1607.5043
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 1293.8083
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 819.5251
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 547.1442
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 539.7277
Epoch 108/10000
1/1 [==============================] - 0s 5ms/step - loss: 854.0928
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 2005.6273
Epo

1/1 [==============================] - 0s 4ms/step - loss: 1359.0444
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 710.8040
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 958.1260
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 944.9661
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 2555.1394
Epoch 200/10000
1/1 [==============================] - 0s 5ms/step - loss: 1418.4873
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 986.9435
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 611.1321
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 837.5106
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 1256.1217
Epoch 205/10000
1/1 [==============================] - 0s 4ms/step - loss: 932.9464
Epoch 206/10000
1/1 [==============================] - 0s 4ms/step - loss: 377.2095
Epoc

1/1 [==============================] - 0s 6ms/step - loss: 954.4944
Epoch 293/10000
1/1 [==============================] - 0s 5ms/step - loss: 835.2634
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 767.4310
Epoch 295/10000
1/1 [==============================] - 0s 5ms/step - loss: 1456.8926
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 785.6037
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 1418.4034
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 875.2903
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 890.4891
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 714.6403
Epoch 301/10000
1/1 [==============================] - 0s 5ms/step - loss: 1072.7504
Epoch 302/10000
1/1 [==============================] - 0s 4ms/step - loss: 2145.8215
Epoch 303/10000
1/1 [==============================] - 0s 4ms/step - loss: 810.6252
Epoc

1/1 [==============================] - 0s 4ms/step - loss: 911.1319
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 485.7246
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 908.8610
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 407.8122
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - loss: 756.8543
Epoch 394/10000
1/1 [==============================] - 0s 5ms/step - loss: 539.1774
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 421.1407
Epoch 396/10000
1/1 [==============================] - 0s 6ms/step - loss: 1289.4025
Epoch 397/10000
1/1 [==============================] - 0s 6ms/step - loss: 829.7921
Epoch 398/10000
1/1 [==============================] - 0s 6ms/step - loss: 1608.3213
Epoch 399/10000
1/1 [==============================] - 0s 7ms/step - loss: 1227.4362
Epoch 400/10000
1/1 [==============================] - 0s 6ms/step - loss: 1098.1748
Epoc

...

1/1 [==============================] - 0s 5ms/step - loss: 411.7602
Epoch 1452/10000
1/1 [==============================] - 0s 5ms/step - loss: 783.0698
Epoch 1453/10000
1/1 [==============================] - 0s 4ms/step - loss: 357.3949
Epoch 1454/10000
1/1 [==============================] - 0s 4ms/step - loss: 483.3275
Epoch 1455/10000
1/1 [==============================] - 0s 5ms/step - loss: 222.1135
Epoch 1456/10000
1/1 [==============================] - 0s 4ms/step - loss: 312.0724
Epoch 1457/10000
1/1 [==============================] - 0s 4ms/step - loss: 800.2310
Epoch 1458/10000
1/1 [==============================] - 0s 5ms/step - loss: 1381.1163
Epoch 1459/10000
1/1 [==============================] - 0s 4ms/step - loss: 555.0849
Epoch 1460/10000
1/1 [==============================] - 0s 4ms/step - loss: 188.3520
Epoch 1461/10000
1/1 [==============================] - 0s 4ms/step - loss: 904.5903
Epoch 1462/10000
1/1 [==============================] - 0s 5ms/step - loss: 430.7

1/1 [==============================] - 0s 5ms/step - loss: 585.9526
Epoch 1548/10000
1/1 [==============================] - 0s 5ms/step - loss: 1708.7188
Epoch 1549/10000
1/1 [==============================] - 0s 4ms/step - loss: 954.3940
Epoch 1550/10000
1/1 [==============================] - 0s 4ms/step - loss: 795.5093
Epoch 1551/10000
1/1 [==============================] - 0s 4ms/step - loss: 990.4998
Epoch 1552/10000
1/1 [==============================] - 0s 5ms/step - loss: 609.0073
Epoch 1553/10000
1/1 [==============================] - 0s 5ms/step - loss: 441.7540
Epoch 1554/10000
1/1 [==============================] - 0s 4ms/step - loss: 462.4044
Epoch 1555/10000
1/1 [==============================] - 0s 4ms/step - loss: 572.2266
Epoch 1556/10000
1/1 [==============================] - 0s 4ms/step - loss: 422.7573
Epoch 1557/10000
1/1 [==============================] - 0s 5ms/step - loss: 802.6777
Epoch 1558/10000
1/1 [==============================] - 0s 5ms/step - loss: 559.8

1/1 [==============================] - 0s 4ms/step - loss: 703.1841
Epoch 1644/10000
1/1 [==============================] - 0s 4ms/step - loss: 688.9874
Epoch 1645/10000
1/1 [==============================] - 0s 5ms/step - loss: 449.7948
Epoch 1646/10000
1/1 [==============================] - 0s 5ms/step - loss: 528.4774
Epoch 1647/10000
1/1 [==============================] - 0s 4ms/step - loss: 722.0859
Epoch 1648/10000
1/1 [==============================] - 0s 5ms/step - loss: 610.3569
Epoch 1649/10000
1/1 [==============================] - 0s 4ms/step - loss: 290.5911
Epoch 1650/10000
1/1 [==============================] - 0s 4ms/step - loss: 333.5150
Epoch 1651/10000
1/1 [==============================] - 0s 4ms/step - loss: 850.4475
Epoch 1652/10000
1/1 [==============================] - 0s 4ms/step - loss: 713.7224
Epoch 1653/10000
1/1 [==============================] - 0s 4ms/step - loss: 1873.8666
Epoch 1654/10000
1/1 [==============================] - 0s 4ms/step - loss: 751.3

1/1 [==============================] - 0s 4ms/step - loss: 896.4515
Epoch 1740/10000
1/1 [==============================] - 0s 4ms/step - loss: 1012.4597
Epoch 1741/10000
1/1 [==============================] - 0s 6ms/step - loss: 476.1090
Epoch 1742/10000
1/1 [==============================] - 0s 5ms/step - loss: 702.5854
Epoch 1743/10000
1/1 [==============================] - 0s 4ms/step - loss: 852.0638
Epoch 1744/10000
1/1 [==============================] - 0s 5ms/step - loss: 798.7061
Epoch 1745/10000
1/1 [==============================] - 0s 4ms/step - loss: 1044.7567
Epoch 1746/10000
1/1 [==============================] - 0s 4ms/step - loss: 911.4293
Epoch 1747/10000
1/1 [==============================] - 0s 5ms/step - loss: 1072.8323
Epoch 1748/10000
1/1 [==============================] - 0s 4ms/step - loss: 453.8895
Epoch 1749/10000
1/1 [==============================] - 0s 4ms/step - loss: 312.5371
Epoch 1750/10000
1/1 [==============================] - 0s 4ms/step - loss: 957

1/1 [==============================] - 0s 4ms/step - loss: 472.5284
Epoch 1836/10000
1/1 [==============================] - 0s 4ms/step - loss: 290.9001
Epoch 1837/10000
1/1 [==============================] - 0s 4ms/step - loss: 298.7707
Epoch 1838/10000
1/1 [==============================] - 0s 4ms/step - loss: 397.0833
Epoch 1839/10000
1/1 [==============================] - 0s 4ms/step - loss: 487.9454
Epoch 1840/10000
1/1 [==============================] - 0s 5ms/step - loss: 394.7355
Epoch 1841/10000
1/1 [==============================] - 0s 4ms/step - loss: 965.9194
Epoch 1842/10000
1/1 [==============================] - 0s 4ms/step - loss: 669.4802
Epoch 1843/10000
1/1 [==============================] - 0s 4ms/step - loss: 1282.5607
Epoch 1844/10000
1/1 [==============================] - 0s 4ms/step - loss: 470.1906
Epoch 1845/10000
1/1 [==============================] - 0s 4ms/step - loss: 955.7839
Epoch 1846/10000
1/1 [==============================] - 0s 4ms/step - loss: 584.3

In [29]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 590ms/step
[826.368](test_target) - [[796.0393]](prediction) = 30.328693359375052


In [30]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.036701195302062825

In [31]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [32]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.300755,-0.527817,-1.242369,-1.294258,-0.985718,-1.173238,-1.018535,-0.977767,1.148205,-1.122043,...,1.308949,1.412346,-1.267049,0.925774,0.402439,-1.372251,1.360201,1.353721,-1.370823,1.394785
1,0.169724,1.400155,0.036043,0.153497,-0.385361,-0.097218,-0.340403,-0.395977,0.140893,-0.184474,...,-0.190804,-0.643250,0.089521,0.462967,-1.375329,0.389992,-0.344854,-0.322487,0.384358,-0.495076
2,1.131031,-0.872338,1.206325,1.140761,1.371079,1.270457,1.358938,1.373744,-1.289098,1.306518,...,-1.118144,-0.769096,1.177528,-1.388742,0.972890,0.982259,-1.015347,-1.031233,0.986465,-0.899709


train_target:


,Piauí - Consumo de Cimento (t)
0,536.766
1,639.535
2,625.022


test_input:


...

,Piauí - IDH Renda,Piauí - Desemprego,Piauí - IDH Educacao,Piauí - IDH Longevidade,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Piauí - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.823567,0.785725,1.126392,-0.598225,0.529639,0.378729,0.253155,0.276615,1.108405,2.635823,...,-1.482779,-2.260447,0.91739,1.011919,0.418013,-0.843464,1.091606,-0.183281,-0.442768,-1.555022


test_target:


,Piauí - Consumo de Cimento (t)
12,826.368


1/1 [==============================] - 1s 578ms/step
Error: 30.328693359375052




[621.436767578125,
 679.1614990234375,
 650.5780029296875,
 742.5107421875,
 799.104248046875,
 824.66845703125,
 826.1080932617188,
 630.1038208007812,
 699.2138671875,
 796.039306640625]

In [33]:
display(targets)
display(predictions)

[687.536,
 778.779,
 755.7138571428571,
 794.9454857142857,
 842.6876571428572,
 831.356,
 626.223,
 702.437,
 788.164,
 826.368]

[621.436767578125,
 679.1614990234375,
 650.5780029296875,
 742.5107421875,
 799.104248046875,
 824.66845703125,
 826.1080932617188,
 630.1038208007812,
 699.2138671875,
 796.039306640625]

In [34]:
mae = mean_absolute_error(predictions, targets)
mae

76.50553818359376

In [35]:
porcentage = mae/np.mean(targets)
porcentage

0.10021408657031147